In [ ]:
#IMPORT DAS BIBLIOTECAS NECESSÁRIAS
import numpy as np #PARA DEFINIR OS PONTOS DE OBSERVAÇÕES 
from GeneticAlgorithm import GeneticAlgorithm as GA #IMPORTAR A CLASSE ALGORITMO GENETICO
import matplotlib.pyplot as plt #PARA VISUALIZAR O RESULTADO
from Models import rect #IMPORTANDO O MODELO
from Data.Functionals import phi #IMPORTANDO A FUNCAO DE AJUSTE
from Data.Population import Fontes #PRA CRIAR A POPULACAO USANDO O MELHOR DEPOIS DO GENETICO

In [ ]:
#DEFINICAO DOS PARAMETROS PRO GENETICO
nobs = 100
xobs = np.linspace(-1000,1000,nobs) #SO PRA CALCULAR O GZ DO MELHOR INDIVIDUO DEPOIS DO GENETICO
zobs = np.zeros( nobs ) #SO PRA CALCULAR O GZ DO MELHOR INDIVIDUO DEPOIS DO GENETICO

# dimensoes do dique vertical:
xmin = -100.
xmax = +100.
zmin = 200.0
zmax = 800.0
rho = 2.95

rec1 = rect( -300, -100, 50, 150, 2 )
rec2 = rect( -200, 0, 150, 250, 2 )
rec3 = rect( -100, 100, 250, 350, 2 )
rec4 = rect( 0, 200, 350, 450, 2 )

models = [ rec1, rec2, rec3, rec4 ]
gz1 = rec1.Gz( xobs, zobs )
gz1 = rec1.addnoise( )

gz2 = rec2.Gz( xobs, zobs )
gz2 = rec2.addnoise( )

gz3 = rec3.Gz( xobs, zobs )
gz3 = rec3.addnoise( )

gz4 = rec4.Gz( xobs, zobs )
gz4 = rec4.addnoise( )


model_gz = gz1 + gz2 + gz3 + gz4 

nfontes = 20 #NUMERO DE BOLINHAS DE CADA INDIVIDUO
min_bounds = [ -1000, 50, 1e6/nfontes ] #ONDE PROCURAR AS SOLUCOES
max_bounds = [ +1000, 1000, 1e12/nfontes ] #ONDE PROCURAR AS SOLUCOES
nind = 50 #NUMERO DE INDIVIDUOS
pmut = 0.1 #PROBABILIDADE DE CADA INDIVIDUO MUTAR
mu = 0.0009 #FATOR DE REGULARIZACAO
ngera = 5000 #NUMERO DE GERACOES

In [ ]:
#ALGORITMO GENETICO
ga = GA( phi, [ model_gz, 'L2', mu ], min_bounds, max_bounds, pmut, nfontes, nind) #CRIANDO O AG USANDO OS PARAMETROS ESCOLHIDOS
ga.Initialize()
ga.FirstFit()
pop = Fontes( ) #CRIANDO UMA NOVA POPULACAO PRA GUARDAR O MELHOR DO GENETICO
melhor_primeira = pop.Gera_from_Existing( [ ga.winner ] ) #GUARDANDO O MELHOR DO GENETICO NESSA NOVA POPULACAO(QUE É UMA CLASSE) ( O MELHOR DO GENETICO TEM O NOME DE ga.winner)
gz_best_primeira = 0 #INICIALIZANDO O GZ DO MELHOR
for b in melhor_primeira: #O MELHOR É UM DICIONARIO, ONDE CADA CHAVE( OBJETO SPHERE ) GUARDA AS PROPIEDADES DESSA SPHERE( X, Y, M)
    for i in b: #AQUI PEGAMOS APENAS AS CHAVES( SPHERES )
        gz_best_primeira += i.Gz(xobs, zobs) #CALCULAMOS O GZ DE TODAS AS SPHERES


In [ ]:
#PLOTANDO A O MODELO, AS BOLINHAS E AS CURVAS
figure, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(18, 11), facecolor='w', sharex=True, 
                                  gridspec_kw={'hspace': 0.0})

error = (model_gz - gz_best_primeira)
ax1.plot(xobs, model_gz, 'r-', label='Gz Observado ', linewidth=3)
ax1.plot(xobs, gz_best_primeira, 'b-', label='Gz Predito ', linewidth=3)
#ax1.plot(xobs, error, 'k-', label='Error')
ax1.legend(fontsize=12)
ax1.set_ylabel('Gz (mGal)', fontsize=15, labelpad=8)
ax1.set_xlim(-750, 750)
ax1.grid()
ax1.label_outer()
ax1.tick_params(axis='both',labelsize = 14)

# ----------------------------------------------------------------------------------------------

ax2.scatter(ga.winner[:, 0], ga.winner[:, 1], c='blue', s=300.0, alpha=0.5)
#ax2.scatter(xobs[0:-1:5], zobs[0:-1:5] + 10, s=20, c='green')

for index,model in enumerate( models ):
    ax2.plot([model.params[0], model.params[0], model.params[1], model.params[1], model.params[0]], \
             [model.params[2], model.params[3], model.params[3], model.params[2], model.params[2]], "-r")
    
    if index == 0:
        ax2.fill_between([model.params[0], model.params[0], model.params[1], model.params[1], model.params[0]], \
                     [model.params[2], model.params[3], model.params[3], model.params[2], model.params[2]], alpha = 0.1, facecolor = 'red',\
                     label='Modelo de escada')
    else:
        ax2.fill_between([model.params[0], model.params[0], model.params[1], model.params[1], model.params[0]], \
                     [model.params[2], model.params[3], model.params[3], model.params[2], model.params[2]], alpha = 0.1, facecolor = 'red')

ax2.legend(fontsize=12 )
ax2.set_ylabel('Profundidade (m)', fontsize=15, labelpad=8)
ax2.set_ylim(0, 1000)
ax2.set_xlabel('Cordenada x (m)', fontsize=15, labelpad=8)
ax2.invert_yaxis()
ax2.label_outer()
ax2.tick_params(axis='both',labelsize = 14)
plt.savefig('sol_inicial_escada.pdf', dpi = 300, bbox_inches = 'tight', transparent = True)

plt.show( )

In [ ]:
ga.start( ngera ) #COMECANDO AS ITERACOES (NESTE CASO, ngera)

pop = Fontes( ) #CRIANDO UMA NOVA POPULACAO PRA GUARDAR O MELHOR DO GENETICO
best = pop.Gera_from_Existing( [ ga.winner ] ) #GUARDANDO O MELHOR DO GENETICO NESSA NOVA POPULACAO(QUE É UMA CLASSE) ( O MELHOR DO GENETICO TEM O NOME DE ga.winner)
gz_best = 0 #INICIALIZANDO O GZ DO MELHOR
for b in best: #O MELHOR É UM DICIONARIO, ONDE CADA CHAVE( OBJETO SPHERE ) GUARDA AS PROPIEDADES DESSA SPHERE( X, Y, M)
    for i in b: #AQUI PEGAMOS APENAS AS CHAVES( SPHERES )
        gz_best += i.Gz(xobs, zobs) #CALCULAMOS O GZ DE TODAS AS SPHERES

thetas = ga.bests_theta #GUARDANDO A LISTA DOS THETAS (REGULARIZACAO = THETA)

In [ ]:
#PLOTANDO A O MODELO, AS BOLINHAS E AS CURVAS
figure, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(18, 11), facecolor='w', sharex=True, gridspec_kw={'hspace': -0.01})

error = np.std(model_gz - gz_best)
ax1.plot(xobs, model_gz, 'r-', label='Gz Observado ', linewidth=3)
ax1.plot(xobs, gz_best, 'b-', label='Gz Predito ', linewidth=3)
ax1.plot(xobs, error, 'k-', label='Erro', linewidth=2)
ax1.legend(fontsize=12)
ax1.set_ylabel('Gz (mGal)', fontsize=15, labelpad=8)
ax1.set_xlim(-750, 750)
ax1.grid()
ax1.label_outer()
ax1.tick_params(axis='both',labelsize = 14)

# -------------------------------------------------------------------------------------------------

ax2.scatter(ga.winner[:, 0], ga.winner[:, 1], c='blue', s=300.0, alpha=0.5)

#ax2.scatter(xobs[0:-1:5], zobs[0:-1:5] + 10, s=20, c='green')

for index,model in enumerate( models ):
    ax2.plot([model.params[0], model.params[0], model.params[1], model.params[1], model.params[0]], \
             [model.params[2], model.params[3], model.params[3], model.params[2], model.params[2]], "-r")
    
    if index == 0:
        ax2.fill_between([model.params[0], model.params[0], model.params[1], model.params[1], model.params[0]], \
                     [model.params[2], model.params[3], model.params[3], model.params[2], model.params[2]], alpha = 0.1, facecolor = 'red',\
                     label='Modelo de escada')
    else:
        ax2.fill_between([model.params[0], model.params[0], model.params[1], model.params[1], model.params[0]], \
                     [model.params[2], model.params[3], model.params[3], model.params[2], model.params[2]], alpha = 0.1, facecolor = 'red')

ax2.legend( fontsize=12 )
ax2.set_ylabel('Profundidade (m)', fontsize=15, labelpad=8)
ax2.set_ylim(0, 1000)
ax2.set_xlabel('Cordenada x (m)', fontsize=15, labelpad=8)
ax2.invert_yaxis()
ax2.label_outer()
ax2.tick_params(axis='both',labelsize = 14)
plt.savefig('best_escada.pdf', dpi = 300, bbox_inches = 'tight', transparent = True)
plt.show( )

In [ ]:
#PLOTANDO A FUNCAO THETA
plt.figure( figsize = ( 8,8 ) )
plt.plot( range( ngera ), thetas, '*-g')
plt.xlabel('Gerações', fontsize=15, labelpad=5)
plt.ylabel(r'$\theta(p)$ ', fontsize=18)
plt.tick_params(axis='both', labelsize=13)
plt.grid()
plt.savefig('Conv_escada.pdf', dpi = 300, bbox_inches = 'tight', transparent = True)
plt.show()


In [ ]:
# Plotting the histogram of residual
from scipy.stats import norm
residuo = model_gz_noised - gz_best
res_mu = np.mean(residuo)
res_sigma = np.std(residuo)
s = (residuo - res_mu) / res_sigma

s = np.ravel(s)
nbins = int(np.max(s) - np.min(s))
n, bins, patches = plt.hist(s, 20, normed = True, facecolor = 'blue', alpha = 1.)
y_hist = norm.pdf(bins, np.mean(s), np.std(s))
plt.plot(bins, y_hist, 'k--', linewidth = 2)
#pyplot.text(3.5,0.25, '$\mu$=%0.5f' % (res_mu), horizontalalignment = 'center', fontsize = 12)
#pyplot.text(3.5,0.3, '$\sigma$=%0.5f' % (res_sigma), horizontalalignment = 'center', fontsize = 12)
plt.xlabel('Residual (nT)' , fontsize = 20, labelpad = 5)
plt.ylabel('Frequency', fontsize = 20, labelpad = 10)
plt.xlim(-5,5)
plt.xticks(np.arange(-5., 5 + 2.5, 2.5), fontsize = 18)
plt.yticks(fontsize = 18)
plt.savefig('hist.pdf', dpi = 300, bbox_inches = 'tight', transparent = True)
plt.show()

In [ ]:
mmm = (massa_verdadeira - ga.winner[0,2]*nfontes)/ massa_verdadeira
mmm